In [1]:
pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.6 MB/s eta 0:00:00


In [3]:
pip install requests pandas


In [10]:
pip install selenium pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

In [25]:

pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00


In [26]:
from google_play_scraper import reviews
import pandas as pd
import time

def kumpulkan_review(id_aplikasi, target_total):
    hasil = []
    token_lanjut = None

    while len(hasil) < target_total:
        batch, token_lanjut = reviews(
            id_aplikasi,
            lang='id', country='id',
            count=200,
            continuation_token=token_lanjut
        )
        hasil += batch
        time.sleep(1.2)

        if token_lanjut is None:
            break

    return hasil[:target_total]

# Konfigurasi aplikasi Tokopedia
id_tokopedia = 'com.tokopedia.tkpd'
label_aplikasi = 'Tokopedia'
jumlah_target = 7000

print(f"📥 Sedang mengunduh {jumlah_target} ulasan dari {label_aplikasi}...")

data_ulasan = []

try:
    daftar_review = kumpulkan_review(id_tokopedia, jumlah_target)
    for r in daftar_review:
        data_ulasan.append({
            'app': label_aplikasi,
            'review': r.get('content', ''),
            'score': r.get('score')
        })
except Exception as err:
    print(f"⚠️ Terjadi error saat mengambil ulasan: {err}")

# Simpan ke CSV
dataframe = pd.DataFrame(data_ulasan)
dataframe.to_csv('data_ulasan_tokopedia_7000.csv', index=False)

print(f"✅ Ulasan berhasil disimpan dalam 'data_ulasan_tokopedia_7000.csv' dengan total {len(dataframe)} entri.")


📥 Sedang mengunduh 7000 ulasan dari Tokopedia...
✅ Ulasan berhasil disimpan dalam 'data_ulasan_tokopedia_7000.csv' dengan total 7000 entri.


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 1. Load data
df = pd.read_csv('data_ulasan_tokopedia_7000.csv')
df.dropna(subset=['review', 'score'], inplace=True)

# 2. Label sentimen
def labelkan(score):
    if score <= 2:
        return 'negatif'
    elif score == 3:
        return 'netral'
    else:
        return 'positif'

df['label'] = df['score'].apply(labelkan)

In [29]:
df.head()

,app,review,score,label
0,Tokopedia,🤣🤣🤣🤣🤣 Mau ganti nomer rekening gopay saja tida...,1,negatif
1,Tokopedia,Saya bulan juni 2025 ada membeli barang tapi t...,1,negatif
2,Tokopedia,tokopedia semua barang Original semuanya,5,positif
3,Tokopedia,ok,5,positif
4,Tokopedia,"Kita pengen tokopedia yg dulu, tokopedia gausa...",1,negatif


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   app     7000 non-null   object
 1   review  7000 non-null   object
 2   score   7000 non-null   int64 
 3   label   7000 non-null   object
dtypes: int64(1), object(3)
memory usage: 218.9+ KB


In [33]:
# 3. TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['review'])
y = df['label']

# 4. Split data 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Train SVM
model_svm = SVC(kernel='linear')
model_svm.fit(X_train, y_train)

# 6. Evaluasi
y_pred = model_svm.predict(X_test)
print("🔍 Evaluasi SVM (TF-IDF + 80:20):")
print(classification_report(y_test, y_pred))


🔍 Evaluasi SVM (TF-IDF + 80:20):
              precision    recall  f1-score   support

     negatif       0.79      0.93      0.86       581
      netral       0.00      0.00      0.00        84
     positif       0.90      0.88      0.89       735

    accuracy                           0.85      1400
   macro avg       0.57      0.60      0.58      1400
weighted avg       0.80      0.85      0.82      1400



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
